In [1]:
#start
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [2]:
date_columns = ['expiration_date', 'registration_init_time']

train_data = pd.read_csv('E:/KKBOX/train.csv')
test_data = pd.read_csv('E:/KKBOX/test.csv', index_col=0)
item_data = pd.read_csv('E:/KKBOX/songs.csv')
user_data = pd.read_csv('E:/KKBOX/members.csv', parse_dates=date_columns)

all_data_org = pd.concat([train_data, test_data])
all_data_org = all_data_org.merge(item_data, on='song_id', how='left')
all_data_org = all_data_org.merge(user_data, on='msno', how='left')
#print(all_data_org[0:6])
#all_data_org.to_csv('E:/KKBOX/KKBOXNo3/all_data_org.csv')

c:\users\tangx\appdata\local\programs\python\python36\lib\site-packages\numpy\lib\arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


                                           msno  \
0  FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=   
1  Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=   
2  Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=   
3  Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=   
4  FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=   
5  FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=   

                                        song_id   source_screen_name  \
0  BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=              Explore   
1  bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=  Local playlist more   
2  JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=  Local playlist more   
3  2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=  Local playlist more   
4  3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=              Explore   
5  3Hg5kugV1S0wzEVLAEfqjIV5UHzb7bCrdBRQlGygLvU=              Explore   

  source_system_tab      source_type  target  song_length genre_ids  \
0           explore  online-playlist     1.0     206471.0       35

In [3]:
enc = LabelEncoder()

for col in [
    'msno', 'song_id', 'source_screen_name', 
    'source_system_tab', 'source_type', 
    'artist_name', 'composer', 'lyricist', 'gender'
]:
    all_data_org[col] = enc.fit_transform(all_data_org[col].fillna('nan'))
    
all_data_org['genre_ids'] = 
    
for col in ['language', 'city', 'registered_via']:
    all_data_org[col] = enc.fit_transform(all_data_org[col].fillna(-2)) 

In [4]:
all_data_org['time'] = all_data_org.index / len(all_data_org)

n = len(train_data)
train_data = all_data_org[:n]
test_data = all_data_org[n:]
print(n)

#train_data.to_hdf('data/train_data.hdf', key='wsdm')
#test_data.to_hdf('data/test_data.hdf', key='wsdm')
#train_data.to_hdf('E:/KKBOX/wsdm-kkboxmusicrec-py/train_data.hdf',key='candy')
#test_data.to_hdf('E:/KKBOX/wsdm-kkboxmusicrec-py/test_data.hdf',key='candy')

7377418


In [5]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

In [7]:
from itertools import combinations

import scipy.sparse as sp
from scipy.sparse import coo_matrix
from lightfm import LightFM

c:\users\tangx\appdata\local\programs\python\python36\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [8]:
date_cols = ['expiration_date', 'registration_init_time']

train_data = pd.read_hdf('E:/KKBOX/wsdm-kkboxmusicrec-py/train_data.hdf', parse_dates=date_cols)
test_data = pd.read_hdf('E:/KKBOX/wsdm-kkboxmusicrec-py/test_data.hdf', parse_dates=date_cols)

In [10]:
all_data = pd.concat([train_data, test_data])

In [11]:
df_test = test_data
df_history_test = train_data

df_trains = []
df_history_trains = []

n = len(test_data)
shift = int(0.05*len(train_data))

for i in range(2):
    m = -i*shift
    if m == 0:
        m = None
    df_trains.append(train_data[-(n + i*shift):m])
    df_history_trains.append(train_data[:-(n + i*shift)])
not_categorical_columns = [
    'target', 
    'song_length', 
    'registration_init_time', 
    'expiration_date', 
    'time', 
    'bd',
]
categorical_columns = all_data.columns.difference(not_categorical_columns)

orders = {}

for col in categorical_columns:
    orders[col] = 10 ** (int(np.log(all_data[col].max() + 1) / np.log(10)) + 1)

orders

{'artist_name': 100000,
 'city': 100,
 'composer': 100000,
 'gender': 10,
 'genre_ids': 1000,
 'language': 100,
 'lyricist': 100000,
 'msno': 100000,
 'registered_via': 10,
 'song_id': 1000000,
 'source_screen_name': 100,
 'source_system_tab': 10,
 'source_type': 100}

In [ ]:
def get_group(df, cols):
    
    group = df[cols[0]].copy()
    for col in cols[1:]:
        group = group * orders[col] + df[col]
        
    return group


def mean(df_history, df, cols):
    
    group = get_group(df, cols)
    group_history = get_group(df_history, cols)
    
    mean_map = df_history.groupby(group_history).target.mean()
    
    return group.map(mean_map).fillna(-1)


def count(df_history, df, cols):
    
    group = get_group(df, cols)
    group_all = get_group(all_data, cols)
    
    count_map = group_all.value_counts()
    
    return group.map(count_map).fillna(0)